In [ ]:
import numpy as np
import pandas as pd
import requests
import holoviews as hv

In [ ]:
hv.extension("bokeh")

# Spectral response

In [ ]:
def bin_spectrum(df, bins):
    bin_assignment = pd.cut(df.index, bins).rename_categories(
        (bins.right + bins.left) / 2
    )
    return df.groupby(bin_assignment).mean()

In [ ]:
spectra_urls = {
    "mCherry": "https://www.fpbase.org/spectra_csv/?q=79,80,158",
    "mCherry2": "https://www.fpbase.org/spectra_csv/?q=1451,1450",
}
spectra = {
    name: pd.read_csv(url)
    .rename(columns={f"{name} {kind}": kind for kind in ("ex", "em", "2p")})
    .set_index("wavelength")
    for name, url in spectra_urls.items()
}

In [ ]:
bins = pd.interval_range(300, 1100, freq=5)

In [ ]:
binned_spectra = {
    name: bin_spectrum(spectrum, bins) for name, spectrum in spectra.items()
}

In [ ]:
(
    hv.Curve(spectra["mCherry"]["em"].dropna())
    * hv.Curve(spectra["mCherry2"]["em"].dropna())
).opts(width=800)

In [ ]:
(
    hv.Curve(binned_spectra["mCherry"]["em"].dropna())
    * hv.Curve(binned_spectra["mCherry2"]["em"].dropna())
).opts(width=800)

In [ ]:
obj_transmission = pd.read_csv("uplxapo_20x.csv", header=None)
obj_transmission.columns = ["wavelength", "transmission"]
obj_transmission.set_index("wavelength", inplace=True)
hv.Curve(obj_transmission).opts(width=800)

In [ ]:
laser_power_shg = pd.read_csv("discovery_vue_shg.csv", header=1).set_index("lambda")
# laser_power_shg.columns = ["power"]
laser_power_pump = pd.read_csv("discovery_vue_pump.csv", header=1).set_index("lambda")
laser_power_pump.columns = ["pump power"]
laser_power = pd.concat(
    [
        laser_power_shg,
        laser_power_pump,
        laser_power_shg.rename(columns={"SHG power": "power"}).append(
            laser_power_pump.rename(columns={"pump power": "power"})
        ),
    ],
    axis=1,
)
laser_power.index.name = "wavelength"
hv.Curve(laser_power, "wavelength", "power").opts(width=800)

In [ ]:
import nd2reader
import matplotlib.pyplot as plt

In [ ]:
nd2 = nd2reader.ND2Reader(
    "/home/jqs1/scratch/191312/G-R_RG/RG_100pct_100ms_100pct_100ms.nd2_0001.nd2"
)

In [ ]:
plt.imshow(nd2.get_frame_2D(t=0))

In [ ]:
import pickle

In [ ]:
with open(
    "/n/groups/paulsson/jqs1/molecule-counting/190521photobleaching_noflatcorr.pickle",
    "rb",
) as f:
    dat = pickle.load(f)

In [ ]:
dat.keys()

In [ ]:
f = np.asarray(
    dat["/n/scratch2/jqs1/190514/GFP_100ms_10pct.nd2"][0]["segmentation_frame"]
)

In [ ]:
plt.imshow(f)